# The $N$-body problem. Maximum: 80 pts

## Problem 0 (Problem statement) 5 pts

Consider the $N$-body problem
$$
    V({\bf y}_j) = \sum_{i=1}^N G({\bf x}_i, {\bf y}_j) q_i, \quad j=1,\dots,N,
$$
where ${\bf x}_i$ is the location of source charges and ${\bf y}_j$ is the location of receivers where the potential $V$ is measured. 
For simplicity in this pset sources and receivers are the same points: ${\bf x}_i = {\bf y}_i$, $i=1,\dots,N$.
The naive summation yields $\mathcal{O}(N^2)$ complexity, which is prohibitive if $N$ is large.
This problem set is devoted to algorithms that break the $\mathcal{O}(N^2)$.


* (5 pts) Name algorithms that break $\mathcal{O}(N^2)$ for $N$-body problem. Specify their complexities. Estimate how much memory and what time requires to estimate all $N$ potentials $V({\bf y}_j)$ for $N=300$ billion particles with naive $\mathcal{O}(N^2)$ summation and $\mathcal{O}(N\log N)$, $\mathcal{O}(N)$ algorithms on a supercomputer
(constants hidden in $\mathcal{O}$ can be found in lectures).

## Problem 1 (The Barnes-Hut algorithm) 35 pts

#### The Barnes-Hut

The Barnes-Hut (BH) idea is quite simple. First, we separate our particles in a quad-tree structure of particle groups. If the group on some tree level is sufficiently far away from a certain particle, we can approximate its potential by using its center of mass. If it is not, we compute its influence recursively by using lower tree levels. The accuracy of the Barnes-Hut algorithm depends on the choise of parameter $\theta = s / d$, where $s$ is the width of the region represented by the internal node, and $d$ is the distance between the body and the node's center-of-mass.

* (6 pts) Propose an algorithm for the quadtree construction. Can you reach $\mathcal{O}(N)$ memory for the storage? Propose a way to store the tree and write the program that computes the tree, given the location of the particles. What do you need to store in each node of the tree?


* (6 pts) Implement Barnes-Hut algorithm. The program should consist of three parts:
    1.  Tree construction given the location of the particles and geometric constant $\theta$  
    2.  Filling the information in the tree (computing the charges and geometric centers)
    3. Computing the product


* (3 pts) Compare the results computed by direct evaluation and Barnes-Hut algorithm. Make sure that you got linear complexity. Study the dependance of accuracy and computational cost on the geometric parameter $\theta$


#### Simplified FMM

In order to break $\log$ term in $\mathcal{O}(N \log N)$ for the Barnes-Hut algorithm a second tree can be used. 
This almost leads us to the FMM algorithm with only one exception: only one term in the multipole expansion is used.

* (15 pts) Now that you are a given a tree from the previous task, code the Barnes-Hut with two trees. The key differences are:
    1. You need to create the interaction list
    2. You also need to build M2L and L2L operators (in standard BH only M2M operator is used)
 
 
* (5 pts) Compare performance and accuracy of the standard and 2-tree BH. Which one is faster?

## Problem 2 (Low-Rank nature of the FMM) (20 pts)

In this problem we will investigate low-rank nature of the interaction between well-separated clusters of particles.

### A. Newton potential

Let us consider Newton potential
$$
    G({\bf x}_i, {\bf y}_j) = \frac{1}{\|{\bf x}_i - {\bf y}_j\|}.
$$
In the following tasks you need to compute the approximate rank of this matrix, by truncating a singular value decomposition 
$$G\approx U\Sigma V^T,$$
so that 
$$\frac{\|G- U\Sigma V^T\|_2}{\|G\|_2} \leqslant 10^{-5}.$$

Let ${\bf x}_j$, $j=1\dots N$ be located randomly within the unit square $[0,1]^2$ and let ${\bf y}_j$, $j=1\dots N$ be randomly located within a speciﬁed target unit square.

* (10 pts) Fix $N=500$. Investigate how the rank of $G$ depends on the distance between boxes of sources and receivers. Provide plottigns. What happens if these boxes have common edge or vertex?

* (5 pts) Make plotting how rank depends on $N$.


### B. High-Frequency issues

Consider Yukawa potential
$$
G({\bf x}_i, {\bf y}_j) = \frac{e^{ik\|{\bf x}_i - {\bf y}_j\|}}{\|{\bf x}_i - {\bf y}_j\|}.
$$

* (5 pts) Fix the position of boxes such that they are well-separated. Plot how the rank of $G$ depends on $k$. What are the diﬃculties presented by having a high value of $k$?

## Problem 3 ($\mathcal{H}$ and $\mathcal{H}^2$ matrices) (20 pts)

Now that we have seen that interaction of charges located in far blocks has low-rank structure, we can move on to $\mathcal{H}$ and $\mathcal{H}^2$ matrices.

#### $\mathcal{H}$ matrices

* Both $\mathcal{H}$ and $\mathcal{H}^2$ matrices consider sources and receivers as groups, so there are 2 trees. This helped us to break the $\log$ term in the total complexity of the FMM. However, for $\mathcal{H}$ matrices the storage and matvec operation is still $\mathcal{O} (N \log N)$. Explain why this happens compared with the FMM.

#### $\mathcal{H}^2$ matrices

Consider charges ${\bf x}_i$ and receivers ${\bf y}_i$, $i=1,\dots,N$ located on a $(0, 1)$ interval. Matrix $G$ is defined as 
$$
G({\bf x}_i, {\bf y}_j) = \frac{1}{\|{\bf x}_i - {\bf y}_j\|}.
$$

Consider 2 levels of hierarchy in matrix $G$ (see the next figure).
<img src="pics/h2.png" alt="Drawing" style="width: 200px;"/>
Here blocks $1$, $2$ and $3$ stand for low-rank blocks and blue blocks are of full rank.
For $\mathcal{H}$ matrices all blocks are treated and stored separatedly. 
$\mathcal{H}^2$ matrices are more complicated.
Namely, rows that produce basis in matrices $1$ and $2$ are assumed to produce a good basis in matrix $3$ (by "good" basis we mean that all other rows can be approximated by these "good" rows with high accuracy). Let us check this property.


* (10 pts) Implement cross approximation method
* (5 pts) Set $N=10 000$. Find numbers of "good" rows in matrices $1$ and $2$ via cross approximation. Stoping tolerance is $10^{-5}$.
* (5 pts) Check that found rows approximate well all rows in $3$. What is the accuracy of this approximation? Compare this accuracy with the case when the same number of rows in $3$ is chosen randomly